![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error

rental_info = pd.read_csv("rental_info.csv")

# Getting the number of rental days
# How to convert into datetime format?
rental_info["return_date"] = pd.to_datetime(rental_info["return_date"])
rental_info["rental_date"] = pd.to_datetime(rental_info["rental_date"])

# How to get number of rental days?
rental_info["rental_length"] = rental_info["return_date"] - rental_info["rental_date"]
rental_info["rental_length_days"] = rental_info["rental_length"].dt.days

# Adding dummy variables using the special features column.
# What is the code to add the dummy variables?
rental_info["deleted_scenes"] = np.where(rental_info["special_features"].str.contains("Deleted Scenes"), 1, 0)
rental_info["behind_the_scenes"] = np.where(rental_info["special_features"].str.contains("Behind the Scenes"), 1, 0)

# Executing a train-test split
# What's the code to do a train-test split?
X = rental_info.drop(["rental_date", "return_date", "special_features", "rental_length", "rental_length_days"], axis=1)
y = rental_info["rental_length_days"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=9)

# Performing feature selection
# Using Lasso Regression
lasso = Lasso(alpha=0.3, random_state=9)
lasso.fit(X_train, y_train)
lasso_coef = lasso.coef_
X_lasso_train, X_lasso_test = X_train.iloc[:, lasso_coef > 0], X_test.iloc[:, lasso_coef > 0]

# Choosing models and performing hyperparameter tuning
# Fitting models to the data
lr = LinearRegression().fit(X_lasso_train, y_train)
dtr = DecisionTreeRegressor(random_state=9)
rfr = RandomForestRegressor()

# Tuning your model
param_dist = {
    'n_estimators': np.arange(1,101,1),
    'max_depth':np.arange(1,11,1)
}
rfr_randomized_search = RandomizedSearchCV(estimator=rfr, param_distributions=param_dist, random_state=9, cv=5)
rfr_randomized_search.fit(X_train, y_train)
rfr_best_params = rfr_randomized_search.best_params_

# Predicting values on test set
# What is an example code to predict values?
y_test_lr_pred = lr.predict(X_lasso_test)
rfr_hyper = RandomForestRegressor(n_estimators=rfr_best_params["n_estimators"],
                                  max_depth=rfr_best_params["max_depth"],
                                  random_state=9)
rfr_hyper.fit(X_train, y_train)
y_test_rfr_hyper_pred = rfr_hyper.predict(X_test)

# Computing mean squared error
# How to compute a model's mean squared error?
mse_lr_lasso = mean_squared_error(y_test, y_test_lr_pred)
mse_rfr_hyper = mean_squared_error(y_test, y_test_rfr_hyper_pred)

best_model = rfr_hyper
best_mse = mse_rfr_hyper